In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import nglview as nv
import ipywidgets as widgets
%matplotlib notebook
import matplotlib.pyplot as plt
from IPython.display import display, display_markdown
from ipywidgets import Layout, HTML
from pathlib import Path

import parmed as pmd
import re

import hublib.use
from hublib.ui import FileUpload, Download
from hublib.cmd import runCommand

from scipy.ndimage import gaussian_filter

import time

%matplotlib notebook
%use gromacs-2018.4
%use pymol-1.8.4

np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)

<IPython.core.display.Javascript object>

In [1]:
cfile = 'complex.gro'
ctop = 'complex.top'

out = !{"gmx editconf -f " + cfile + " -o newbox.gro -bt cubic -d 1"}
out = !{"cp "}
out = !{"gmx grompp -maxwarn 1 -f em.mdp -c newbox.gro -p " + ctop + ' -o em.tpr'}
for line in out:
    print(line)
    
out = !{"gmx mdrun -v -deffnm em"}
for line in out:
    print(line)
    
out = !{"gmx editconf -f em.tpr -mead -o charge.pdb"}
out = !{"gmx trjconv -f em.gro -s em.tpr -o em.pdb"}
out = !{"bash parse_pqr.sh mead.pqr"}

/bin/bash: gmx: command not found
/bin/bash: gmx: command not found
